# CASE STUDY: FASHION CLASS CLASSIFICATION

# STEP 01: PROBLEM STATEMENTAND BUSINESS CASE

Fashion training set consist of 70,000 images divided into 60,000 training and 10,000 testing samples. Dataset sample consist of 28x28 graysacel imamge, associeated with a label from 10 classes

The classes area as follows:
- 0 => Shirt/Top
- 1 => Trouser
- 2 => Pullover
- 3 => Dress
- 4 => Coat
- 5 => Sandal
- 6 => Shirt
- 7 => Sneaker
- 8 => Bag
- 9 => Ankle Boot

Each image is 28 pixels in heigh and 28 in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel with higher numers meaning darker.

# STEP 02: IMPORTING THE DATA


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# read csv file 
fashion_train_db = pd.read_csv('../data/Fashion_mnist_datasets/fashion-mnist_train.csv', sep=',')
fashion_test_db = pd.read_csv('../data/Fashion_mnist_datasets/fashion-mnist_test.csv', sep=',')

# STEP 03: DATA VISUALIZATION

## Eplore the trainning data Set

In [ ]:
fashion_train_db.tail()

In [ ]:
fashion_train_db.head()

In [ ]:
fashion_train_db.shape

## Eplore the testing Data Set

In [ ]:
fashion_test_db.head()

In [ ]:
fashion_test_db.tail()

In [ ]:
fashion_test_db.shape

In [ ]:
training = np.array(fashion_train_db, dtype='float32')

In [ ]:
testing = np.array(fashion_test_db, dtype='float32')

## Visualize an image

In [ ]:
import random
i = random.randint(1, 60000)
plt.imshow(training[i,1:].reshape(28,28))
# now we will display the label
label = training[i, 0]
print("category: {0}".format(label.__str__()))

## View the data in a matrix form

In [ ]:
# Lets view more images in a grid format
# Define the dimensions of the plot grid
W_grid = 15
L_grid = 15

# fig, axes, = plt.subplots(L_grid, W_grid)
# subplot returns the figures object and axes object
# we can use the axes object to plot specific figures at various locations
fig, axes, = plt.subplots(L_grid, W_grid, figsize=(17,17))
axes = axes.ravel() # flaten the 15 x 15 matrix into 225 array

n_training = len(training) # get the lenght of the training dataset


# Select a random number from 0 to n_training
for i in np.arange(0, L_grid * W_grid): # create evenly spaces variables
    # Select a random number
    index = np.random.randint(0, n_training)
    # read and display and image with the selected index
    axes[i].imshow(training[index, 1:].reshape((28,28)))
    axes[i].set_title(training[index, 0], fontsize=8)
    axes[i].axis('off')
plt.subplots_adjust(hspace=0.4)

# STEP 04: TRAINNING THE MDOEL

In [ ]:
x_train = training[:, 1:]/255
y_train = training[:, 0]

In [ ]:
x_test = testing[:, 1:]/255
y_test = testing[:, 0]

In [ ]:
# validation dataset to use during trainning to make a generalize model
from sklearn.model_selection import train_test_split

In [ ]:
# use the train splits 
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2, random_state=12345)

In [ ]:
# take our training, testing and validation to reshape
x_train = x_train.reshape(x_train.shape[0], *(28, 28, 1))
x_test = x_test.reshape(x_test.shape[0], *(28, 28, 1))
x_validate = x_validate.reshape(x_validate.shape[0], *(28, 28, 1))

In [ ]:
# now we will be using keras
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

In [ ]:
# run cnn model 
cnn_model = Sequential()

In [ ]:
# specifiy the number of kernels
cnn_model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation='relu'))

In [ ]:
# specify pooling layers
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
# flat our model
cnn_model.add(Flatten())

In [ ]:
# Specify dimensions
cnn_model.add(Dense(output_dim=32, activation="relu"))

In [ ]:
cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [ ]:
epochs = 50

In [ ]:
cnn_model.fit(x_train,
              y_train,
              batch_size=512,
              nb_epoch=epochs,
              verbose=1,
              validation_data=(x_validate, y_validate))

# Step 05: EVALUATING THE MODEL

In [ ]:
evaluation = cnn_model.evaluate(x_test, y_test)
print("Test Accuracy: {:.3f}".format(evaluation[1]))

In [ ]:
# Use confusion matrix
predicted_classes = cnn_model.predict_classes(x_test)
predicted_classes

In [ ]:
L = 5
W = 5
fig, axes = plt.subplots(L, W, figsize=(12,12))
axes = axes.ravel() # 

for i in np.arange(0,L * W):
    axes[i].imshow(x_test[i].reshape(28,28))
    axes[i].set_title("Prediction Class = {:0.01f}\n True Class =  {:0.01f}".format(predicted_classes[i],y_test[i]))
    axes[i].axes('off')
plt.subplots_adjust(w_space=0.5)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(14, 10))
sns.heatmap(cm, annot=True)
# Sum diagonal element to get the total of tru correct values

In [ ]:
from sklearn.metrics import classification_report
num_classes = 10
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_test, predicted_classes, target_names=target_names))